In [1]:
# %% Initial setup
import pyodbc
import pandas as pd
import warnings
from pathlib import Path
from datetime import timedelta
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score, f1_score
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle

warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")

In [2]:
# Read in the full meds data from using the Meds.sql query in edw_queries
meds = pd.read_csv(Path(r'PATH TO MEDS DATA'))

In [3]:
# Establish SQL server connection
SERVER = 'INSERT SERVER NAME'
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER,
                      Trusted_Connection='Yes')

conn.autocommit = True
cursor = conn.cursor()

# SQL for getting date information
dates_query='''
            SET NOCOUNT ON
            select * from FSM_SCRIPT.fsm_script_dm.basic_endpoints
            '''
dates_df = pd.read_sql(dates_query, conn)

In [4]:
# Filter out undesired routes
routes_to_remove = ['Left Eye', 'Right Eye', 'Topical', 'Nasal', 'Each Eye', 'Both Eyes', 
                    'Vaginal', 'Each Nare', 'Ophthalmic', 'Swish/Spit', 'Swish/Swallow', 
                    'Oral Swab', 'Topical (*)', 'Each Affected Eye', 'Mouth/Throat', 
                    'IntraNasally', 'Right Ear', 'Left Ear', 'Intranasal', 'IntraVITREAL - Right Eye', 
                    'IntraVITREAL - Left Eye', 'Each Ear', 'Both Ears', 'Ocular Insert', 'Operative Eye', 'Inhalation', 'Dental', 'scalp']

filtered_meds = meds[~meds['route'].isin(routes_to_remove)]

# Create a boolean mask for rows to keep
mask = ~(
    filtered_meds['order_display_name'].str.contains('topical', case=False, na=False) |
    filtered_meds['medication_name'].str.contains('topical', case=False, na=False) |
    filtered_meds['generic_name'].str.contains('topical', case=False, na=False)
)

# Apply the mask to filter the DataFrame
filtered_meds = filtered_meds[mask]

In [5]:
# Filter for only meds in the 6 months prior to SCRIPT admission

# Merge the dataframes on the patient ID columns
merged_df = pd.merge(filtered_meds, dates_df, left_on='ir_id', right_on='patient_ir_id', how='left')

# Convert datetime columns to datetime type if they're not already
datetime_columns = ['order_placed_datetime', 'order_start_datetime', 'admission_datetime']
for col in datetime_columns:
    merged_df[col] = pd.to_datetime(merged_df[col])

# Calculate the date 6 months prior to admission
merged_df['six_months_prior'] = merged_df['admission_datetime'] - timedelta(days=180)

# Filter the dataframe
filtered_result = merged_df[
    ((merged_df['order_placed_datetime'] >= merged_df['six_months_prior']) & 
     (merged_df['order_placed_datetime'] <= merged_df['admission_datetime'])) |
    ((merged_df['order_start_datetime'] >= merged_df['six_months_prior']) & 
     (merged_df['order_start_datetime'] <= merged_df['admission_datetime']))
]

# Select only the columns from the original filtered_meds dataframe
filtered_meds = filtered_result[filtered_meds.columns]

In [6]:
# Identify all usage of the medications of interest

# Azathioprine
azathioprine_variations = ['azathioprine', 'imuran', 'azasan']
azathioprine = filtered_meds[
    filtered_meds['medication_name'].str.lower().str.contains('|'.join(azathioprine_variations), na=False) |
    filtered_meds['generic_name'].str.lower().str.contains('|'.join(azathioprine_variations), na=False) |
    filtered_meds['order_display_name'].str.lower().str.contains('|'.join(azathioprine_variations), na=False)
]

# Cyclosporine
cyclosporine_variations = ['cyclosporine', 'neoral', 'sandimmune', 'gengraf']
cyclosporine = filtered_meds[
    filtered_meds['medication_name'].str.lower().str.contains('|'.join(cyclosporine_variations), na=False) |
    filtered_meds['generic_name'].str.lower().str.contains('|'.join(cyclosporine_variations), na=False) |
    filtered_meds['order_display_name'].str.lower().str.contains('|'.join(cyclosporine_variations), na=False)
]

# Cytoxan (Cyclophosphamide)
cytoxan_variations = ['cytoxan', 'cyclophosphamide', 'neosar', 'procytox']
cytoxan = filtered_meds[
    filtered_meds['medication_name'].str.lower().str.contains('|'.join(cytoxan_variations), na=False) |
    filtered_meds['generic_name'].str.lower().str.contains('|'.join(cytoxan_variations), na=False) |
    filtered_meds['order_display_name'].str.lower().str.contains('|'.join(cytoxan_variations), na=False)
]

# Mycophenolate
mycophenolate_variations = ['mycophenolate', 'mycophenolic', 'cellcept', 'myfortic']
mycophenolate = filtered_meds[
    filtered_meds['medication_name'].str.lower().str.contains('|'.join(mycophenolate_variations), na=False) |
    filtered_meds['generic_name'].str.lower().str.contains('|'.join(mycophenolate_variations), na=False) |
    filtered_meds['order_display_name'].str.lower().str.contains('|'.join(mycophenolate_variations), na=False)
]

# Rituximab
rituximab_variations = ['rituximab', 'rituxan', 'truxima', 'ruxience', 'riabni']
rituximab = filtered_meds[
    filtered_meds['medication_name'].str.lower().str.contains('|'.join(rituximab_variations), na=False) |
    filtered_meds['generic_name'].str.lower().str.contains('|'.join(rituximab_variations), na=False) |
    filtered_meds['order_display_name'].str.lower().str.contains('|'.join(rituximab_variations), na=False)
]

# Tacrolimus
tacrolimus_variations = ['tacrolimus', 'prograf', 'advagraf', 'astagraf', 'envarsus', 'hecoria']
tacrolimus = filtered_meds[
    filtered_meds['medication_name'].str.lower().str.contains('|'.join(tacrolimus_variations), na=False) |
    filtered_meds['generic_name'].str.lower().str.contains('|'.join(tacrolimus_variations), na=False) |
    filtered_meds['order_display_name'].str.lower().str.contains('|'.join(tacrolimus_variations), na=False)
]

# Part 2, all the med dataframes are ready, do the analysis

In [7]:
# 1) Join to pt_study_id information
# 2) Make a function which takes the dataframe and a number and creates a new dataframe where it just has 
#    unique pt_study_id in one column and 1s in the other column called med_pred where there would be a 1 if there were more than number dates
def filter_study_ids(df, num, med):

    # Load in the pt_study_id values
    query_enr = ''' 
        select distinct patient_ir_id, pt_study_id
        from FSM_SCRIPT.fsm_script_dm.basic_endpoints'''
    # This dataframe will have IR ID, study ID
    pt_study_id_df = pd.read_sql(query_enr, conn)

    df = pd.merge(left = df, right = pt_study_id_df, left_on = 'ir_id', right_on = 'patient_ir_id', how = 'left')

    # Count the frequency of patient_study_id
    counts = df['pt_study_id'].value_counts()
    # Filter study_ids that appear at least 'num' times
    study_ids = counts[counts >= num].index.tolist()
    # Create a new dataframe
    new_df = pd.DataFrame({'study_id': study_ids, f'{med}': 1})
    return new_df

In [8]:
# See documentation for generate_cm in ICD_identifier (similar but for meds)
def generate_cm(df, threshold, immune_condition, gold_label):

    prediction_df = filter_study_ids(df, threshold, immune_condition)
    prediction_df['study_id'] = prediction_df['study_id'].astype('int64')

    # Load in the IC label data
    # Query for SCRIPT 1.0 patients
    redcap='''
            SET NOCOUNT ON
            drop table if exists #imc
            select * 
            into #imc 
            from (
            select distinct pt_study_id,type_immunocomp
            from FSM_SCRIPT.fsm_script_redcap_dm.redcap_PROJECTID_demographics
            ) x

            select pt_study_id,
                case when type_immunocomp like '%Acute leukemia%' then 1 end as Leukemia,
                case when type_immunocomp like '%Azathioprine%' then 1 end as Azathioprine,
                case when type_immunocomp like '%Chronic corticosteroids%' then 1 end as Chronic_corticosteroids,
                case when type_immunocomp like '%Cyclosporine%' then 1 end as Cyclosporine,
                case when type_immunocomp like '%Cytoxan%' then 1 end as Cytoxan,
                case when type_immunocomp like '%HIV%' then 1 end as HIV,
                case when type_immunocomp like '%Immunoglobulin deficiency%' then 1 end as Immunoglobulin_deficiency,
                case when type_immunocomp like '%Lymphoma%' then 1 end as Lymphoma,
                case when type_immunocomp like '%Mycophenolate (MMF)%' then 1 end as Mycophenolate,
                case when type_immunocomp like '%Multiple myeloma%' then 1 end as Myeloma,
                case when type_immunocomp like '%Myelosuppressive chemotherapy%' then 1 end as Myelosuppressive_chemo,
                case when type_immunocomp like '%Rituximab%' then 1 end as Rituximab,
                case when type_immunocomp like '%Solid organ transplant%' then 1 end as SOT,
                case when type_immunocomp like '%Stem cell transplant%' then 1 end as Stem_cell_transplant,
                case when type_immunocomp like '%Tacrolimus%' then 1 end as Tacrolimus
            from #imc
            '''
    
    # Same query as above but for SCRIPT 2 patients
    redcap2='''
            SET NOCOUNT ON
            drop table if exists #imc2
            select * 
            into #imc2
            from (
            select distinct record_id,emr_ic_type
            from FSM_SCRIPT.fsm_script_redcap_dm.redcap_PROJECTID_emr_info
            ) x

            select record_id as pt_study_id,
                case when emr_ic_type like '%Acute leukemia%' then 1 end as Leukemia,
                case when emr_ic_type like '%Azathioprine%' then 1 end as Azathioprine,
                case when emr_ic_type like '%Chronic corticosteroids%' then 1 end as Chronic_corticosteroids,
                case when emr_ic_type like '%Cyclosporine%' then 1 end as Cyclosporine,
                case when emr_ic_type like '%Cytoxan%' then 1 end as Cytoxan,
                case when emr_ic_type like '%HIV%' then 1 end as HIV,
                case when emr_ic_type like '%Immunoglobulin deficiency%' then 1 end as Immunoglobulin_deficiency,
                case when emr_ic_type like '%Lymphoma%' then 1 end as Lymphoma,
                case when emr_ic_type like '%Mycophenolate (MMF)%' then 1 end as Mycophenolate,
                case when emr_ic_type like '%Multiple myeloma%' then 1 end as Myeloma,
                case when emr_ic_type like '%Myelosuppressive chemotherapy%' then 1 end as Myelosuppressive_chemo,
                case when emr_ic_type like '%Rituximab%' then 1 end as Rituximab,
                case when emr_ic_type like '%Solid organ transplant%' then 1 end as SOT,
                case when emr_ic_type like '%Stem cell transplant%' then 1 end as Stem_cell_transplant,
                case when emr_ic_type like '%Tacrolimus%' then 1 end as Tacrolimus
            from #imc2
            '''
    # This dataframe will have study ID and then every immunosuppressive condition listed above and named as above
    labels_df_1 = pd.read_sql(redcap, conn)
    labels_df_1['pt_study_id'] = labels_df_1['pt_study_id'].astype(int)

    # Same dataframe as above but for SCRIPT 2 patients
    labels_df_2 = pd.read_sql(redcap2, conn)
    labels_df_2['pt_study_id'] = labels_df_2['pt_study_id'].astype(int)

    # Put together the labels for SCRIPT 1 and SCRIPT 2
    labels_df = pd.concat([labels_df_1, labels_df_2], ignore_index=True)

    # Right join prediction_df to labels_df to put predictions and labels together
    final_df = pd.merge(left = prediction_df, right = labels_df, left_on = 'study_id', right_on = 'pt_study_id', how = 'right')
    final_df = final_df.fillna(0)

    # Make sure the types match for confusion matrix input
    final_df[gold_label] = final_df[gold_label].astype(int)
    final_df[immune_condition] = final_df[immune_condition].astype(int)

    # Create and display the confusion matrix
    cm = confusion_matrix(final_df[gold_label], final_df[immune_condition])

    # Calculate metrics
    tn, fp, fn, tp = cm.ravel()
    sensitivity = recall_score(final_df[gold_label], final_df[immune_condition])
    specificity = tn / (tn + fp)
    ppv = precision_score(final_df[gold_label], final_df[immune_condition])
    npv = tn / (tn + fn)
    accuracy = accuracy_score(final_df[gold_label], final_df[immune_condition])
    f1 = f1_score(final_df[gold_label], final_df[immune_condition])

    # Draw confusion matrix (uncomment as needed)
    cm1 = cm[::-1, ::-1]

    plt.figure(figsize=(6, 6))
    sns.heatmap(cm1, annot=True, fmt='d', cmap='Purples',
              xticklabels=['ICD 1', 'ICD 0'],
              yticklabels=['REDCap 1', 'REDCap 0'])
    plt.title(f'Confusion Matrix - {immune_condition}')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    # plt.savefig(f'confusion_matrix_{immune_condition}_{code_frequency}.png')
    # plt.close()

    # Create DataFrame with metrics
    metrics_df = pd.DataFrame({
        'Comparison': [immune_condition],
        'True Negative': [tn],
        'False Positive': [fp],
        'False Negative': [fn],
        'True Positive': [tp],
        'Sensitivity': [sensitivity],
        'Specificity': [specificity],
        'PPV': [ppv],
        'NPV': [npv],
        'Accuracy': [accuracy],
        'F1 Score': [f1]
    })

    return final_df, metrics_df



In [ ]:
# Compile all metrics for one instance of a medication order

med_dfs = [azathioprine, cyclosporine, cytoxan, mycophenolate, rituximab, tacrolimus]
pred_list = ['azathioprine_pred', 'corticosteroids_pred', 'cyclosporine_pred', 'cytoxan_pred', 'mycophenolate_pred', 'myelosuppressive_chemotherapy_pred', 'rituximab_pred', 'tacrolimus_pred']
gold_list = ['Azathioprine', 'Chronic_corticosteroids', 'Cyclosporine', 'Cytoxan', 'Mycophenolate', 'Myelosuppressive_chemo', 'Rituximab', 'Tacrolimus']

# Initialize an empty list to store individual metric DataFrames
metrics_list = []

for med_df, pred, gold in zip(med_dfs, pred_list, gold_list):
    # Generate metrics for each prediction
    extra_df, metrics_df = generate_cm(med_df, 1, pred, gold, is_corrections_applied=True, only_include_llm_patients=True)
    # Append the metrics DataFrame to the list
    metrics_list.append(metrics_df)

# Concatenate all individual metric DataFrames into a single DataFrame
final_metrics_df = pd.concat(metrics_list, ignore_index=True)

# Save metrics as a CSV in this directory for subsequent figure generation
final_metrics_df.to_csv(Path(f'./metrics.csv'), index = False)